### Mapping Data

The "basemap" library provides a not-so-simple facility for geospatial presentations. 
(Installing it is a bit of a chore, and did not go smoothly for me.  I don't have a recipe, other than "Ingredients: 1 wall, 1 head -- Bang head against wall until tender.")

The code here data selected from a dataframe onto a map of the US.  (So it assumes that we are looking at the US database.)  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dtim
from mpl_toolkits.basemap import Basemap
%matplotlib inline

The next two cells are copied from "DailyExp."  This isn't terribly good software practices, but the intent here is to be self-contained and to show all the steps of the process.  You have to break a few eggs to make an omelette. 

In [ ]:
database_url = "https://raw.github.com/CSSEGISandData/COVID-19/master/csse_covid_19_data"

In [ ]:
def fetchDatabase(report_name, remove_cols = {"FIPS", "Lat", "Long_", "UID", "ISO3"}):
    # Find yesterday's date.  The CSSE database is updated daily
    yesterday = dtim.datetime.now() - dtim.timedelta(days=1)
    mmddyyyy = "%02d-%02d-%4d" % (yesterday.month, yesterday.day, yesterday.year)
    
    # build the report filename
    report_url = "%s/csse_covid_19_%s/%s.csv" % (database_url, report_name, mmddyyyy)
    rep = pd.read_csv(report_url)
    # drop a few of the less useful columns
    orig_cols = set(list(rep))
    # make sure we don't try to remove a column that doesn't exist
    actual_remove_cols = list(orig_cols.intersection(remove_cols))
    return rep.drop(columns = actual_remove_cols)

In [ ]:
def doMapPlot(database, column_name):
    fig = plt.figure(figsize=(15,15))
    plt.title(column_name, fontsize=16)
    map2 = Basemap(resolution='l',llcrnrlon=-119,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=49, projection='lcc', lat_1=32, lat_2=45, lon_0=-95)
    map2.drawstates()
    map2.drawcountries(linewidth=1.5, color='black')
    map2.drawcoastlines(linewidth=1.5, color='black')
    lats,lons = (np.array(database["Lat"]), np.array(database["Long_"]))
    stat = database[column_name]
    scale = 400 / np.max(stat)

    map2.scatter(lons, lats, s = scale * stat, latlon=True, marker='D', color='m', zorder=0)
    plt.show()
    return map2

In [ ]:
daily_rep_us = fetchDatabase("daily_reports_us", remove_cols = {})

In [ ]:
daily_rep_us["Deaths_vs_Outcomes"] = np.divide(daily_rep_us["Deaths"], np.add(daily_rep_us["Deaths"], daily_rep_us["Recovered"]))
daily_rep_us["Population (M)"] = 0.1 * np.divide(daily_rep_us["People_Tested"],daily_rep_us["Testing_Rate"])
daily_rep_us['Deaths per Thousand'] = 0.001 * np.divide(daily_rep_us["Deaths"], daily_rep_us["Population (M)"])

In [ ]:
doMapPlot(daily_rep_us, "Deaths per Thousand")